# **Objetivo del Cuaderno Jupyter**

- Desempaquetar las columnas anidadas de `movies_dataset.csv` y traer datos de `actores.parquet` y `directores.parquet`, con el fin de consolidar todas las `posibles` variables predictoras en un solo dataframe. Al final de este cuaderno Jupyter obtendremos una tabla con la estructura que muestra la imagen<br>

<img src='assets/movies_empezar_modelar.png'><br>

**Nota**: Los campos sombreados amarillo son los que desanidaremos en este cuaderno y basicamente quedaran con Ids de sus respectivos campos cuya información estará en otras tablas.  Los campos que están sombreados de verde son campos nuevos para el dataset, producto de las modificaciones en los Cuadernos Jupyter anteriores o la información de Directores y actores añadidas al final de este cuaderno.

- `Para poder dejar` de manera numerica la mayoría de los campos que modificamos o incluímos, `tenemos que guardar esos datos` en algún lado, por sí se necesitan mas adelante, por mi persona o cualquier persona que necesite realizar análisis de estos datos.  Abajo esta un esquema gráfico de las tablas que se crearon EN ESTE cuaderno.<br>

<img src='assets/tablas_nuevas_cuaderno_3.png'><br>


### A tomar en cuenta:
- No es un EDR de TODA la Data, le faltaría las tablas de Actores y Directores y sus respectivas tablas intermedias, ya que tanto actores como directores, supondrían relaciones de N:N contra movies, además, de otros datos que se encuentran en cast.parquet y crew.parquet...  
- Simplemente es una manera de saber donde encontrar la información de todo lo que se desanida en este Cuaderno por sí se necesita mas adelante, o lo necesita otra persona.<br>

In [4]:
import polars as pl

### Función `return_string_without_doubles` 
- Recibe cadenas con pequeños errores de sintaxis que dificultan la labor de `eval` y los corrige <br>
- Leer Docstring para entender su comportamineto

In [1]:
def return_string_without_doubles(cadena:str) -> str:
    """ return_string_without_doubles: Recibe una cadena que representa una lista de diccionarios
                                       (con errores sutiles, que rompen la funcion eval() de python)
                                        y los corrige. Buscando asi obtener Toda la Data posible
                                       
    Nota: En este caso Tanto comentario en codigo lo puede volver confuso. Preferí una documentación mas 
          completa en el docstring. Aposté (como siempre) por nombres de variables explicativos, sobre todo
          en aquellas del tipo `bool` que uso mucho en los condicionales, tratando de que el codigo sea lo 
          mas pythonico posible, para su mejor comprensión.                                                                
    
    Que Hace el Codigo: Por favor preste atención a los ejemplos explicados en las proximas lineas,
                        esto dará una comprensión de que hace el codigo 
    
     EJEMPLO 1   
     RECIBE:   [{\'name\': \'Canal+\', \'id\': 5358}, {\'name\': \'Zespól Filmowy ""Tor""\', \'id\': 7984}]                     
     ERROR:   (El error son las " que rodean a ""Tor"")
     DEVUELVE: [{'name': 'Canal+', 'id': 5358}, {'name': 'Zespól Filmowy  Tor  ', 'id': 7984}] 

     EJEMPLO 2
     RECIBE:   [{\'name\': \'Procirep\', \'id\': 311}, {\'name\': ""Club d\'Investissement Média"", \'id\': 8170}]                                    
     ERROR:    (El error es ""Club d\'Investissement Média"")
     DEVUELVE: [{'name': 'Procirep', 'id': 311}, {'name': 'Club d`Investissement Média', 'id': 8170}]                                      
     NOTA: la comilla que hace papel de apostrofe en d'Investissement  la cambio por un backtick ` despues 
           en un Dataframe puedo volver a reemplazar o dejarlo asi. Obvio que aca tambien el problema era las "" 
                             

    Variables:
    index: va recorriendo la linea
    two_double_open: si es True, estoy corrigiendo algo como lo del ejemplo 2
    single_open: si es True y aparece una "  estoy en precencia de un caso como el ejemplo 1                    

    Args:
        cadena (str): String que representa una lista de Diccionarios

    Returns:
        str: String que representa una lista de Diccionarios Corregida
    """
    cadena_list = list(cadena)
    # La variable de abajo Me indica si hay un minibloque entre "" abierto   
    two_double_open = False 
    # La variable de abajo Me indica si hay un minibloque entre ' abierto
    single_open = False      
    for index in range(len(cadena_list)):
        if cadena_list[index] == '"' and not two_double_open and not single_open:
            if cadena_list[index+1] == '"':
                cadena_list[index] = " "
                cadena_list[index+1] = "'"
                two_double_open = True
                marcador = index
        elif cadena_list[index] == "'" and not two_double_open and not single_open:
            single_open = True
        elif cadena_list[index] == "'" and single_open:
            single_open = False
        elif cadena_list[index] == '"' and single_open:
            cadena_list[index] = ' '        
        elif cadena_list[index] == "'" and two_double_open and index > (marcador + 1):
            cadena_list[index] = '`'
        elif cadena_list[index] == '"' and two_double_open:
            cadena_list[index] = "'"
            cadena_list[index+1] = " "
            two_double_open = False
        elif cadena_list[index] == '\\':
            cadena_list[index] = ' '
    cadena_return = ''.join(cadena_list[:])
    return cadena_return 

### Probando la Funcion `return_string_without_doubles`

In [3]:
cadena_enviada = '[{\'name\': \'Canal+\', \'id\': 5358}, {\'name\': \'Zespól Filmowy ""Tor""\', \'id\': 7984}]'
cadena_recibida = return_string_without_doubles(cadena_enviada)

cadena_enviada_2 = '[{\'name\': \'Procirep\', \'id\': 311}, {\'name\': ""Club d\'Investissement Média"", \'id\': 8170}]'
cadena_recibida_2  = return_string_without_doubles(cadena_enviada_2)

print(cadena_recibida)
print(cadena_recibida_2)

[{'name': 'Canal+', 'id': 5358}, {'name': 'Zespól Filmowy   Tor  ', 'id': 7984}]
[{'name': 'Procirep', 'id': 311}, {'name':  'Club d`Investissement Média' , 'id': 8170}]


## **IMPORTANTE**
Necesito el archivo `movies.parquet` tal cual fue procesado en el Cuaderno Anterior.  La celda de abajo lo que hace es traer una copia de es archivo, por sí no logró ejecutar por alguna razón todas las celdas de los anteriores cuadernos Jupyter

In [2]:
import gdown

url = 'https://drive.google.com/uc?export=download&id=1fRagZ5PKQce8RPnCb-9jAR6IXebJmi5K'

gdown.download(url, 'data/movies.parquet', quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=1fRagZ5PKQce8RPnCb-9jAR6IXebJmi5K
To: d:\BOOTCAMP-HENRY\MODULO-07-LABS\data\movies.parquet
100%|██████████| 8.45M/8.45M [00:01<00:00, 7.54MB/s]


'data/movies.parquet'

In [5]:
movies = pl.read_parquet('data/movies.parquet')
movies.head()

belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return,release_year,release_month,release_day_of_week,name_of_month,name_of_day
str,f32,str,i32,str,str,f32,str,str,date,f32,f32,str,str,str,str,f32,i32,f32,i32,i8,i8,str,str
"""{'id': 10194, 'name': 'Toy Sto…",3e7,"""[{'id': 16, 'name': 'Animation…",862,"""en""","""Led by Woody, Andy's toys live…",21.946943,"""[{'name': 'Pixar Animation Stu…","""[{'iso_3166_1': 'US', 'name': …",1995-10-30,3.73554048e8,81.0,"""[{'iso_639_1': 'en', 'name': '…","""Released""","""""","""Toy Story""",7.7,5415,12.451801,1995,10,1,"""OCTUBRE""","""LUNES"""
"""""",6.5e7,"""[{'id': 12, 'name': 'Adventure…",8844,"""en""","""When siblings Judy and Peter d…",17.015539,"""[{'name': 'TriStar Pictures', …","""[{'iso_3166_1': 'US', 'name': …",1995-12-15,2.62797248e8,104.0,"""[{'iso_639_1': 'en', 'name': '…","""Released""","""Roll the dice and unleash the …","""Jumanji""",6.9,2413,4.043035,1995,12,5,"""DICIEMBRE""","""VIERNES"""
"""{'id': 119050, 'name': 'Grumpy…",0.0,"""[{'id': 10749, 'name': 'Romanc…",15602,"""en""","""A family wedding reignites the…",11.7129,"""[{'name': 'Warner Bros.', 'id'…","""[{'iso_3166_1': 'US', 'name': …",1995-12-22,0.0,101.0,"""[{'iso_639_1': 'en', 'name': '…","""Released""","""Still Yelling. Still Fighting.…","""Grumpier Old Men""",6.5,92,0.0,1995,12,5,"""DICIEMBRE""","""VIERNES"""
"""""",1.6e7,"""[{'id': 35, 'name': 'Comedy'},…",31357,"""en""","""Cheated on, mistreated and ste…",3.859495,"""[{'name': 'Twentieth Century F…","""[{'iso_3166_1': 'US', 'name': …",1995-12-22,8.145216e7,127.0,"""[{'iso_639_1': 'en', 'name': '…","""Released""","""Friends are the people who let…","""Waiting to Exhale""",6.1,34,5.09076,1995,12,5,"""DICIEMBRE""","""VIERNES"""
"""{'id': 96871, 'name': 'Father …",0.0,"""[{'id': 35, 'name': 'Comedy'}]""",11862,"""en""","""Just when George Banks has rec…",8.387519,"""[{'name': 'Sandollar Productio…","""[{'iso_3166_1': 'US', 'name': …",1995-02-10,7.6578912e7,106.0,"""[{'iso_639_1': 'en', 'name': '…","""Released""","""Just When His World Is Back To…","""Father of the Bride Part II""",5.7,173,0.0,1995,2,5,"""FEBRERO""","""VIERNES"""


***
### **Desempaquetar todas las Columnas con información Múltiple**

In [5]:
# Nota: id tambien lo Tomo porque necesito vincular los datos extraidos con el `id` del movie al que pertenecen
id_movies_list = movies['id'].to_list() # Lista con todos los id del dataframe movies

# Lista con todos los production_companies del dataframe movies
prod_companies_list = movies['production_companies'].to_list() 

# Lista con todos los production_countries del dataframe movies
prod_countries_list = movies['production_countries'].to_list() 

# Lista con todos los belongs_to_collection del dataframe movies
belongs_to_collec_list = movies['belongs_to_collection'].to_list() 

# Lista con todos los genres del dataframe movies
genres_list = movies['genres'].to_list() 

# Lista con todos los idiomas
spoken_languages_list = movies['spoken_languages'].to_list()  

## **Creando Dataframe `languages` y `languages_movies`** 

In [6]:
data = []
se_rompio = False
for i in range(len(spoken_languages_list)):
    try:
        # cadena = return_string_without_doubles(spoken_languages_list[i])
        cadena = spoken_languages_list[i] 
        lista = eval(cadena)
        for diccionario in lista:
            diccionario['movie_id'] = id_movies_list[i]
            data.append(diccionario)
    except:
        se_rompio = True
        print(f"se rompio en {i}")
        break

if not se_rompio:
    languages_movies = pl.DataFrame(data=data)

"""
    # La variable `se_rompio` junto con el indice de la lista que esta recorriendo el `for` es una combinacion de 
    # depuración del codigo.  PUDE HABERLA QUITADO, despues de estar seguro que ya el codigo corría,  prefiero
    # dejarla, porque explica la logica de trabajo
"""

In [7]:
languages_movies.head(1)

iso_639_1,name,movie_id
str,str,i64
"""en""","""English""",862


Se Observan Informacion faltante en el campo name de `languages`

In [8]:
mask = languages_movies['name'] == ""
print(languages_movies.filter(mask)['iso_639_1'].to_list())

['gd', 'mn', 'mk', 'kw', 'nv', 'mi', 'tl', 'gd', 'yi', 'mi', 'tl', 'yi', 'yi', 'yi', 'ne', 'km', 'gd', 'yi', 'km', 'yi', 'iu', 'mi', 'sh', 'bo', 'gd', 'ty', 'yi', 'gd', 'yi', 'gd', 'si', 'bo', 'as', 'sh', 'yi', 'gn', 'gd', 'gd', 'yi', 'mk', 'bo', 'iu', 'nv', 'nv', 'yi', 'yi', 'yi', 'lo', 'gd', 'yi', 'xh', 'yi', 'gd', 'bo', 'yi', 'yi', 'cr', 'yi', 'sh', 'sh', 'tl', 'iu', 'nv', 'yi', 'gd', 'ku', 'hy', 'oc', 'oc', 'ku', 'bo', 'to', 'ce', 'qu', 'ku', 'mi', 'nv', 'yi', 'mk', 'yi', 'tl', 'am', 'km', 'ce', 'iu', 'yi', 'ku', 'sh', 'gd', 'tg', 'bo', 'gd', 'sh', 'bo', 'qu', 'hy', 'sh', 'yi', 'tl', 'tt', 'yi', 'se', 'tl', 'hy', 'ml', 'sh', 'mi', 'co', 'yi', 'mn', 'tl', 'yi', 'yi', 'tl', 'ku', 'dz', 'tl', 'yi', 'hy', 'ku', 'sh', 'hy', 'yi', 'ce', 'yi', 'gd', 'xh', 'yi', 'bo', 'ku', 'am', 'ht', 'yi', 'sh', 'xh', 'ln', 'ku', 'tg', 'mn', 'mn', 'sh', 'bo', 'yi', 'ce', 'xh', 'mn', 'my', 'mn', 'km', 'yi', 'sa', 'mn', 'sh', 'ce', 'bo', 'sa', 'ku', 'fy', 'tk', 'tt', 'tl', 'ku', 'ml', 'km', 'ny', 'st', 'xh

In [9]:
# Me preparo un dicionario con la informaicon faltante
faltantes = {'mr': 'Marathi',
             'tl': 'Tagalog', 
             'iu': 'Inuktitut',
             'as': 'Assamese',
             'lo': 'Lao',
             'se': 'Northern Sami',
             'mi': 'Maori',
             'dz': 'Dzongkha',
             'qu': 'Quechua',
             'sc': 'Sardinian',
             'tg': 'Tajik',
             'sa': 'Sanskrit',
             'my': 'Burmese',
             'ln': 'Lingala',
             'cr': 'Cree',
             'to': 'Tonga',
             'gu': 'Gujarati',
             'co': 'Corsican',
             'ht': 'Haitian Creole',
             'ne': 'Nepali',
             'ty': 'Tahitian',
             'sg': 'Sango', 
             'jv': 'Javanese',
             'ug': 'Uighur',
             'fo': 'Faroese',
             'si': 'Sinhalese',
             'ce': 'Chechen',
             'st': 'Sotho, Southern',
             'km': 'Central Khmer',
             'xh': 'Xhosa',
             'mk': 'Macedonian',
             'ku': 'Kurdish',		
             'sn': 'Shona',
             'bo': 'Tibetan',
             'kw': 'Cornish',
             'ki': 'Kikuyu',
             'ny': 'Chichewa',
             'sm': 'Samoan',
             'gn': 'Guaraní',
             'tt': 'Tatar',
             'lb': 'Luxembourgish',
             'br': 'Breton',
             'ay': 'Aymara',
             'mh': 'Marshallese',
             'gd': 'Scottish Gaelic',
             'nv': 'Navajo',
             'tk': 'Turkmen',
             'tn': 'Tswana',
             'ml': 'Malayalam',
             'oc': 'Occitan',
             'ig': 'Igbo',
             'ab': 'Abkhazian',
             'mn': 'Mongolian',
             'am': 'Amharic',
             'yi': 'Yiddish',
             'hy': 'Armenian',
             'bi': 'Bislama',
             'fy': 'Western Frisian', 
             'sh': 'Serbocroata'}

# Cuando algun registro tenga en su columna `iso_639_1` el valor que corresponda a la key de la respectiva iteracion
# entonces en la columna `name` coloco la informacion que tenga value de lo contrario, que deje lo que habia en `name``
for key, value in faltantes.items():
    languages_movies = languages_movies.with_columns([
        pl.when(pl.col("iso_639_1") == key)
          .then(pl.lit(value))
          .otherwise(pl.col("name"))
          .alias("name")
])

Para una mejor Normalizacion necesito crear un id numerico a languages

In [10]:
languages = languages_movies[['iso_639_1', 'name']] 

languages = languages.unique() # Elimino Valores repetidos

languages = languages.with_row_index(name='id', offset=1)  # Creo un indice

print(languages.shape[0])

languages.head()

133


id,iso_639_1,name
u32,str,str
1,"""ja""","""日本語"""
2,"""cr""","""Cree"""
3,"""hu""","""Magyar"""
4,"""si""","""Sinhalese"""
5,"""la""","""Latin"""


Ahora necesito que `languages_movies` obtenga esa columna `id` que tengo en `languages`.  Maneras de hacerlo:<br>
- Utilizando un Join donde deben coincidir las columnas `iso_639_1` de ambos Dataframe
- Utilizando un  `ciclo for` donde se recorre `languajes`, y se filtra `languajes_movies` por `iso_639_1`

In [11]:
languages_movies = languages_movies.with_columns(pl.lit(0).alias('id'))  # A languages le creo columna `id` y le coloco 0 a todo

for i in range(languages.shape[0]):
    iso_639_1 = languages[i,1]  # la Columna 1 representa `iso_639_1`
    id = languages[i,0]          # la Columna 0 representa el `id`   
    languages_movies = languages_movies.with_columns(
        pl.when(pl.col('iso_639_1') == iso_639_1)
          .then(pl.lit(id))
          .otherwise(pl.col('id'))
          .alias('id')
    )

languages_movies.head()

iso_639_1,name,movie_id,id
str,str,i64,i32
"""en""","""English""",862,20
"""en""","""English""",8844,20
"""fr""","""Français""",8844,80
"""en""","""English""",15602,20
"""en""","""English""",31357,20


## **Creando DataFrame `companies` y `companies_movies`**

In [12]:
data = []
se_rompio = False
for i in range(len(prod_companies_list)):
    try:
        cadena = return_string_without_doubles(prod_companies_list[i])
        lista = eval(cadena)
        for diccionario in lista:
            diccionario['movie_id'] = id_movies_list[i]
            data.append(diccionario)
    except:
        se_rompio = True
        print(f"se rompio en {i}")
        break

if not se_rompio:
    companies_movies = pl.DataFrame(data=data)

companies_movies.head(1)


name,id,movie_id
str,i64,i64
"""Pixar Animation Studios""",3,862


In [13]:
companies = companies_movies.drop('movie_id').unique()

# **Creando Dataframe `countries` y `countries_movies`**

In [14]:
data = []
se_rompio = False
for i in range(len(prod_countries_list)):
    try:
        cadena = return_string_without_doubles(prod_countries_list[i])
        lista = eval(cadena)
        for diccionario in lista:
            diccionario['movie_id'] = id_movies_list[i]
            data.append(diccionario)
    except:
        se_rompio = True
        print(f"se rompio en {i}")
        break

if not se_rompio:
    countries_movies = pl.DataFrame(data=data)

countries_movies.head(1)    

iso_3166_1,name,movie_id
str,str,i64
"""US""","""United States of America""",862


Para una mejor Normalizacion necesito crear un id numerico a countries

In [15]:
countries = countries_movies[['iso_3166_1', 'name']] 

countries = countries.unique() # Elimino Valores repetidos

countries = countries.with_row_index(name='id', offset=1)  # Creo un indice

print(countries.shape[0])

countries.head()

161


id,iso_3166_1,name
u32,str,str
1,"""KW""","""Kuwait"""
2,"""VN""","""Vietnam"""
3,"""CD""","""Congo"""
4,"""PY""","""Paraguay"""
5,"""LK""","""Sri Lanka"""


Ahora necesito que `countries_movies` obtenga esa columna `id` que tengo en `countries`.  Maneras de hacerlo<br>
- Utilizando un Join donde deben coincidir las columnas `iso_3166_1` de ambos Dataframe
- Utilizando un  `ciclo for` donde se recorre `countries`, y se filtra `countries_movies` por `iso_3166_1`

In [16]:
countries_movies = countries_movies.with_columns(pl.lit(0).alias('id'))  # A countries le creo columna `id` y le coloco 0 a todo

for i in range(countries.shape[0]):
    iso_3166_1 = countries[i,1]  # la Columna 1 representa `iso_3166_1`
    id = countries[i,0]          # la Columna 0 representa el `id`   
    countries_movies = countries_movies.with_columns(
        pl.when(pl.col('iso_3166_1') == iso_3166_1)
          .then(pl.lit(id))
          .otherwise(pl.col('id'))
          .alias('id')
    )

countries_movies.head()

iso_3166_1,name,movie_id,id
str,str,i64,i32
"""US""","""United States of America""",862,96
"""US""","""United States of America""",8844,96
"""US""","""United States of America""",15602,96
"""US""","""United States of America""",31357,96
"""US""","""United States of America""",11862,96


## **Creando Dataframe `genres` y `genres_movies`**

In [17]:
data = []
se_rompio = False
for i in range(len(genres_list)):
    try:
        cadena = return_string_without_doubles(genres_list[i])
        lista = eval(cadena)
        for diccionario in lista:
            diccionario['movie_id'] = id_movies_list[i]
            data.append(diccionario)
    except:
        se_rompio = True
        print(f"se rompio en {i}")
        break

if not se_rompio:
    genres_movies = pl.DataFrame(data=data)

In [18]:
genres_movies.head(1)

id,name,movie_id
i64,str,i64
16,"""Animation""",862


In [22]:
genres = genres_movies.drop('movie_id').unique()
genres.head(1)

id,name
i64,str
99,"""Documentary"""


## **Creando el Dataframe `Collections` y `collections_movies`** 

In [24]:
data = []
se_rompio = False
for i in range(len(belongs_to_collec_list)):
    try:
        cadena = belongs_to_collec_list[i]
        if cadena != '':
            cadena = return_string_without_doubles(cadena)
            diccionario = eval(cadena)
            diccionario['movie_id'] = id_movies_list[i]
            data.append(diccionario)
    except:
        se_rompio = True
        print(f"se rompio en {i}")
        break

if not se_rompio:
    collections_movies = pl.DataFrame(data=data)

collections_movies.head(1)    

id,name,poster_path,backdrop_path,movie_id
i64,str,str,str,i64
10194,"""Toy Story Collection""","""/7G9915LfUQ2lVfwMEEhDsn3kT4B.j…","""/9FBwqcd9IRruEDUrTdcaafOMKUq.j…",862


In [25]:
collections = collections_movies.drop('movie_id').unique()
collections.head(1)

id,name,poster_path,backdrop_path
i64,str,str,str
361140,"""House of Angels Collection""","""/o2wfpVSaixGLo5SYx9f6OXmKZTG.j…","""/wjpF8AxTNBGBVO4XpbdBYytB4uU.j…"


***
Ya tengo: <br>
- languages
- companies
- countries
- genres
- collections

Ahora necesito que en los campos de `movies` asociados a estas variables aparezca una lista con los distintos id de ese renglon al que esta asociado.<br>
**Ejemplo**: En el campo `genres` dentro de `movies` aparecerá una lista solo con los `id` de los generos a que está asociada esta pelicula


In [29]:
"""
Ubicacion de Las Columnas Dentro de `movies`
id                     --> Columna 3

original_language      --> Columna 4
spoken_languages       --> Columna 12
belongs_to_collection  --> Columna 0
genres                 --> Columnas 2
production_companies   --> Columna 7
production_countries   --> Columna 8

"""
# Recorro uno a uno las peliculas
for i in range(movies.shape[0]):
    movie_id = movies[i,3]
    

    # Trabajando en la Columna original_language o columna 4 del dataframe
    if movies[i,4] != '':
        # ACA Filtro a languages por el `iso_639_1` del registro de movies donde estoy parado en esta iteracion
        # extraigo el `id` del lenguaje y lo asigno en movies['original_languages'] 
        mask = languages['iso_639_1'] == movies[i,4]
        movies[i,4] = str(languages.filter(mask)['id'][0])

    # Trabajando en la Columna spoken_languages o columna 12 del dataframe
    if movies[i,12] != '':
        mask = languages_movies['movie_id'] == movie_id
        # ACA igual a la logica del `if` anterior, pero aca son multiples resultados a la hora de filtrar
        # por eso lo paso a una lista.
        movies[i,12] = str(languages_movies.filter(mask)['id'].to_list())    

    # Trabajando en la Columna belongs_to_collection o columna 0 del dataframe
    if movies[i,0] != '':
        mask = collections_movies['movie_id'] == movie_id
        movies[i,0] = str(collections_movies.filter(mask)['id'].to_list())

    # Trabajando en la Columna genres o columna 2 del dataframe
    if movies[i,2] != '':
        mask = genres_movies['movie_id'] == movie_id
        movies[i,2] = str(genres_movies.filter(mask)['id'].to_list())

    # Trabajando en la Columna production_companies o columna 7 del dataframe 
    if movies[i,7] != '':
        mask = companies_movies['movie_id'] == movie_id
        movies[i,7] = str(companies_movies.filter(mask)['id'].to_list())
        
    # Trabajando en la Columna production_countries o columna 8 del dataframe
    if movies[i,8] != '':
        mask = countries_movies['movie_id'] == movie_id
        movies[i,8] = str(countries_movies.filter(mask)['id'].to_list())

In [30]:
movies.head(1)

belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return,release_year,release_month,release_day_of_week,name_of_month,name_of_day
str,f32,str,i32,str,str,f32,str,str,date,f32,f32,str,str,str,str,f32,i32,f32,i32,i8,i8,str,str
"""[10194]""",3e7,"""[16, 35, 10751]""",862,"""20""","""Led by Woody, Andy's toys live…",21.946943,"""[3]""","""[96]""",1995-10-30,3.73554048e8,81.0,"""[20]""","""Released""","""""","""Toy Story""",7.7,5415,12.451801,1995,10,1,"""OCTUBRE""","""LUNES"""


Ahora dentro de `movies` la columna `original_language` se `casteara a Numerico`

In [31]:
# Primero debo sustituir los ""  por "0" para evitar errores al castear
movies = movies.with_columns(
    pl.when(pl.col('original_language') == "")
      .then(pl.lit("0"))
      .otherwise(pl.col('original_language'))
      .alias('original_language')
)

# Ahora paso la columna a Numerico
movies = movies.with_columns(pl.col('original_language').cast(pl.Int32).alias('original_language'))


In [32]:
movies.head(1)

belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return,release_year,release_month,release_day_of_week,name_of_month,name_of_day
str,f32,str,i32,i32,str,f32,str,str,date,f32,f32,str,str,str,str,f32,i32,f32,i32,i8,i8,str,str
"""[10194]""",3e7,"""[16, 35, 10751]""",862,20,"""Led by Woody, Andy's toys live…",21.946943,"""[3]""","""[96]""",1995-10-30,3.73554048e8,81.0,"""[20]""","""Released""","""""","""Toy Story""",7.7,5415,12.451801,1995,10,1,"""OCTUBRE""","""LUNES"""


***
Toca quedarme solo con los campos `id` y `movies_id` en las tablas 
- `languages_movies`
- `companies_movies`
- `countries_movies`
- `genres_movies`
- `collections_movies`


In [33]:
languages_movies = languages_movies[['id', 'movie_id']]
companies_movies = companies_movies[['id', 'movie_id']]
countries_movies = countries_movies[['id', 'movie_id']]
genres_movies = genres_movies[['id', 'movie_id']]
collections_movies = collections_movies[['id', 'movie_id']]

***
Toca crear una Tabla movies especial para el datawarehouse y guardar las tablas

In [38]:
borrar_columnas = ['belongs_to_collection', 'genres', 'production_companies', 'production_countries',
                   'spoken_languages', 'release_month', 'release_year', 'release_day_of_week', 'name_of_month',
                   'name_of_day']
movies_normalizado = movies.drop(borrar_columnas)

**ESTAS SON LAS TABLAS QUE APARECEN EN LA SEGUNDA IMAGEN MOSTRADA AL PRINCIPIO DE ESTE CUADERNO**

In [39]:
languages.write_parquet('data/datawarehouse/languages.parquet')
languages_movies.write_parquet('data/datawarehouse/languages_movies.parquet')

collections.write_parquet('data/datawarehouse/collections.parquet')
collections_movies.write_parquet('data/datawarehouse/collections_movies.parquet')

genres.write_parquet('data/datawarehouse/genres.parquet')
genres_movies.write_parquet('data/datawarehouse/genres_movies.parquet')

companies.write_parquet('data/datawarehouse/companies.parquet')
companies_movies.write_parquet('data/datawarehouse/companies_movies.parquet')

countries.write_parquet('data/datawarehouse/countries.parquet')
countries_movies.write_parquet('data/datawarehouse/countries_movies.parquet')

movies_normalizado.write_parquet('data/datawarehouse/movies_normalizado.parquet')

***
## **Falta meter inf de Actores y Directores en Movies para modelar**

**Nota:** Ya tenemos las tablas del datawarehouse (ya salvamos la información de manera ordenada), PERO para modelar necesitamos una tabla `movies` con todas las posibles variables predictoras incluidas. A la tabla `movies` que ya tenemos, falta incluirle los `id` de los `Directores` y `Actores` que estuvieron involucrados en cada pelicula, ya que esto es un factor a tomar en cuenta en los gustos de los usuarios, y por ende también es información valiosa para el modelo de recomendación, busque similitudes

Añadiendo los `id` de `director` y `actores` asociados a cada película  en `movies`


- Traeré a dataframes las tablas de `directores.parquet` y `actores.parquet` para rellenar los campos.
- Le añadiré una columna de `director` y otra de `actores`

**NOTA**: la celda de abajo lo que hace es descargar los archivos `actores.parquet` y `directores.parquet` 

In [6]:
url1 = 'https://drive.google.com/uc?export=download&id=1xHBWDozPK2u_ZzapEbZyUx1SSIlRDCDN'
url2 = 'https://drive.google.com/uc?export=download&id=1e5fmvkZfcIzqw-fPNZTgf5YNskI_Yh4J'

gdown.download(url1, 'data/actores.parquet', quiet=False)
gdown.download(url2, 'data/directores.parquet', quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=1xHBWDozPK2u_ZzapEbZyUx1SSIlRDCDN
To: d:\BOOTCAMP-HENRY\MODULO-07-LABS\data\actores.parquet
100%|██████████| 4.02M/4.02M [00:03<00:00, 1.09MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1e5fmvkZfcIzqw-fPNZTgf5YNskI_Yh4J
To: d:\BOOTCAMP-HENRY\MODULO-07-LABS\data\directores.parquet
100%|██████████| 388k/388k [00:00<00:00, 3.85MB/s]


'data/directores.parquet'

In [7]:
actores = pl.read_parquet('data/actores.parquet')
directores = pl.read_parquet('data/directores.parquet')

In [44]:
movies = movies.with_columns([
    pl.lit(0).alias('director'),
    pl.lit('[]').alias('actores')
])

In [45]:
"""
en la tabla `actores` 
id_actor          --> columna 0
movies            --> columna 2  --> esta es la lista de peliculas a las que esta asociado el actor

en la tabla `directores` 
id_director       --> columna 0
movies            --> columna 2  --> esta es la lista de peliculas a las que esta asociado el director


en la tabla `movies`
id                --> columna 3
director          --> columna 24
actores           --> columna 25
"""
# Añadiendo los id de Directores a las Peliculas
for i in range(directores.shape[0]):
    id_director = directores[i,0]
    id_movies_list = directores[i,2].to_list()

    # Asignando el id_director de esta Iteracion a las pelicula que corresponda
    movies = movies.with_columns(
        pl.when(pl.col('id').is_in(id_movies_list))
          .then(pl.lit(id_director))
          .otherwise(pl.col('director'))
          .alias('director')
    )

### **LA EJECUCION DEL CODIGO DE ABAJO DURA MUCHO TENER PRECAUCION**

In [46]:
# PRECAUCION CON ESTE BLOQUE DE CODIGO SON DEMASIADAS ITERACIONES
# TOMAR EN CUENTA QUE SON MAS DE 200000 ACTORES Y 45000 PELICULAS Y EN CADA PELICULA HAY VARIOS ACTORES
# AGARRA A TODOS LOS ACTORES Y LOS AÑADE A LA LISTA QUE HAY EN LA COLUMNA `actores`, 
# LA VENTAJA: CONSOLIDAR EN UN SOLO DATAFRAME TODA LAS POSIBLES VARIABLES IMPORTANTES PARA LA PREDICCION
for i in range(actores.shape[0]):
    id_actor = actores[i,0]
    id_movies_list = actores[i,2].to_list()

    for id_movie in id_movies_list:
        for index in range(movies.shape[0]):
            if movies[index,3] == id_movie:
                # si es Tipo str uso eval de tener algo alli que a pesar de ser un campo de Un(1) reconoce
                # como pl.series entonces que aplique el metodo .to_list()  --> Cosas de Python y Polars
                actor_list = eval(movies[index,25]) if type(movies[index,25]) == str else movies[index,25].to_list()
                actor_list.append(id_actor)
                movies[index,25] = str(actor_list)
                break  

##### **Guardando la tabla `movies_modelar`** 

In [141]:
movies.write_parquet('data/movies_modelar.parquet')